# Childrens Act Disposals

In [1]:
# libraries
import pandas as pd
import pydbtools as pydb
import boto3
from datetime import datetime
#import re

In [2]:
# Structured  dataframes
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 900)
pd.set_option("display.max_colwidth", 200)

In [3]:
# Assigning databases and S3 bucket
db1 = "familyman_dev_v2"
fcsq_db = "fcsq"
s3 = boto3.resource("s3")
bucket = s3.Bucket("alpha-family-data")
snapshot_date = "2021-08-19"

### Extracting disposals 

In [13]:
# The first select query extracts orders where the type of order is defined by the event model.
# The second select query extracts disposals where the type of order is determined by the value of the field model. Included in this are orders of no order (ORDNOM), orders refused (ORDREF), and 
#        orders withdrawn (G63). C43 contains child arrangement orders (contact/residence - the most common orders types in children act cases) 
d1 = f""" 
SELECT 
  event,
  case_number,
  event_model,
  receipt_date,
  entry_date,
  error,
  creating_court,
  CAST(NULL AS VARCHAR) AS field_model,
  CAST(NULL AS VARCHAR) AS all_ord_types,
  CAST(NULL AS VARCHAR) AS split_ord_type_value
FROM 
  {db1}.events 
WHERE 
  ((event_model IN ('C23','C25','C26','C27','C28','C29',
                        'C30','C31','C32A','C32B','C33','C34A','C34B','C35A','C35B','C37',
                        'C38A','C38B','C39','C40','C42','C43A','C44A','C44B','C45A','C45B',
                        'C53','C80','C81','C82') 
            AND error = 'N')
    OR (event_model = 'CPA' AND error = 'N' AND narrative Is Null))
  AND mojap_snapshot_date = date '{snapshot_date}'


UNION ALL


SELECT 
  e.event,
  e.case_number,
  e.event_model,
  e.receipt_date,
  e.entry_date,
  e.error,
  e.creating_court,
  f.field_model,
  f.value as all_ord_types,
  TRIM(ord_type) as split_ord_type_value
FROM 
  {db1}.events e
  INNER JOIN {db1}.event_fields f
    ON e.event = f.event
  CROSS JOIN UNNEST(SPLIT(f.value,',')) AS t(ord_type)  
WHERE 
  ((f.field_model IN ('C21_3', 'C24_6', 'C36_5', 'C43_5', 'ORDNOM_5', 'ORDREF_5', 'G63_1'))
    AND error = 'N')
  AND e.mojap_snapshot_date = date '{snapshot_date}'
  AND f.mojap_snapshot_date = date '{snapshot_date}'
  
"""

pydb.create_temp_table (d1,"ca_disps")

In [21]:
pydb.read_sql_query ("select count(*) count from __temp__.ca_disps where extract (year from receipt_date) > 2010")

,count
0,5624322


### Extracting children

In [27]:
d2 = f"""
  SELECT 
    a.*,
    f.value children,
    TRY_CAST(TRIM(child_role_id) as bigint) child_role_id
  FROM 
    __temp__.ca_disps a
    LEFT JOIN {db1}.event_fields f
      ON f.event = a.event
   CROSS JOIN UNNEST(SPLIT(f.value,',')) AS t(child_role_id)
  WHERE 
     f.field_model IN('C21_CH','C23_CH','C24_CH','C25_CH','C26_CH','C27_CH','C28_CH','C29_CH',
                      'C30_CH','C31_CH','C32A_CH','C32B_CH','C33_CH','C34A_CH',
                      'C34B_CH','C35A_CH','C35B_CH','C36_CH','C37_CH','C38A_CH',
                      'C38B_CH','C39_CH','C40_CH','C42_CH','C43_CH',
                      'C43A_CH','C44A_CH','C44B_CH','C45A_CH','C45B_CH','C53_CH',
                      'C80_CH','C81_CH','C82_CH','CPA_CH','ORDNOM_CH','ORDREF_CH', 'G63_CH')
    AND child_role_id <> ''
    AND f.mojap_snapshot_date = DATE'{snapshot_date}'
"""

pydb.create_temp_table (d2,"ca_disps_child_event")

In [29]:
pydb.read_sql_query ("select * from __temp__.ca_disps_child_event limit 20")

,event,case_number,event_model,receipt_date,entry_date,error,creating_court,field_model,all_ord_types,split_ord_type_value,children,child_role_id
0,20200073853,GH03P00262,C21,2003-10-31,2003-10-31 08:34:27,N,GH,C21_3,A,A,20200024130,20200024130
1,20301509889,GL19P00567,C21,2019-12-05,2019-12-10 12:08:51,N,GL,C21_3,A,A,10609813,10609813
2,20800185315,GG04P00573,C21,2005-02-14,2005-02-16 15:36:11,N,GG,C21_3,A,A,20800054471,20800054471
3,20800187702,GG05P00183,C21,2005-03-15,2005-03-16 11:57:30,N,GG,C21_3,A,A,20800060354,20800060354
4,20801061006,GG14P00637,C21,2015-12-02,2015-12-09 10:56:25,N,GG,C21_3,A,A,5470091,5470091
5,21100291998,GU06P00354,C21,2006-12-08,2006-12-12 09:00:35,N,GU,C21_3,A,A,21100074179,21100074179
6,21100576952,NZ12C00115,C21,2012-11-28,2012-12-20 17:29:20,N,GU,C21_3,A,A,2975518,2975518
7,21101426315,GU21C00067,C21,2021-05-21,2021-05-28 10:00:30,N,GU,C21_3,A,A,12021069,12021069
8,21500191539,HL08C10030,C21,2008-09-30,2008-10-01 10:18:07,N,HL,C21_3,A,A,21500060394,21500060394
9,21500221131,HL10P00026,C21,2010-03-17,2010-03-18 09:22:58,N,HL,C21_3,A,A,21500069784,21500069784


In [30]:
d3 = f"""
  SELECT
    a.*,
    r.role child_role_id,
    p.dob,
    p.gender,
    r.delete_flag
  FROM
    __temp__.ca_disps a
    JOIN {db1}.roles r 
      on a.case_number = r.case_number
    JOIN {db1}.parties p 
      on r.party = p.party 
  WHERE
    event not in (SELECT event FROM __temp__.ca_disps_child_event)
    AND role_model in ('CHLDC', 'CHLDZ')
    AND r.mojap_snapshot_date = DATE'{snapshot_date}'
    AND p.mojap_snapshot_date = DATE'{snapshot_date}'
"""

pydb.create_temp_table(d3, "ca_disps_child_case")

In [31]:
pydb.read_sql_query ("select * from __temp__.ca_disps_child_case limit 20")

,event,case_number,event_model,receipt_date,entry_date,error,creating_court,field_model,all_ord_types,split_ord_type_value,child_role_id,dob,gender,delete_flag
0,21600407883,HS12P00505,C43,2014-07-10,2014-08-08 12:27:45,N,HS,C43_5,"CAL, CAST",CAST,3101821,2006-02-23,2,N
1,21600407883,HS12P00505,C43,2014-07-10,2014-08-08 12:27:45,N,HS,C43_5,"CAL, CAST",CAL,3101821,2006-02-23,2,N
2,12704685965,BM13P08223,C43,2013-11-27,2013-12-19 09:13:23,N,BM,C43_5,CN,CN,3469154,2003-04-06,2,N
3,12704514897,BM13P08223,C43,2013-05-30,2013-06-05 11:22:54,N,BM,C43_5,CN,CN,3469154,2003-04-06,2,N
4,55500054665,UV13P00202,C21,2013-07-22,2013-07-25 10:45:26,N,SA,C21_3,C,C,3865968,2004-07-21,1,N
5,52600021790,TF13P03333,C43,2013-09-26,2013-10-07 15:52:19,N,ST,C43_5,IC,IC,4070277,2004-12-29,2,N
6,47700153442,LX13C02264,C21,2013-11-25,2013-12-02 16:23:04,N,LV,C21_3,A,A,4294323,2006-09-04,1,N
7,46900330273,IL13C00559,C21,2013-08-02,2013-08-06 11:15:04,N,IL,C21_3,C,C,4566148,2013-12-25,2,N
8,24902223359,LN14P00082,C21,2014-11-12,2014-11-18 15:10:18,N,LN,C21_3,A,A,4569805,2003-05-21,1,N
9,14100981855,BD13P00482,C43,2013-03-22,2013-03-22 13:47:10,N,BD,C43_5,PS,PS,3572596,2012-11-05,2,N


In [33]:
d4 = f"""
  SELECT 
    a.case_number,
    a.receipt_date,
    a.year,
    a.quarter,
    a.event,
    a.field_model,
    a.court_code,
    a.order_type,
    a.child_role_id,
    p.dob,
    p.gender,
    r.delete_flag
  FROM
    __temp__.ca_disps_child_event a
    JOIN {db1}.roles r on a.child_role_id = r.role
    JOIN {db1}.parties p on r.party = p.party
  WHERE
    r.mojap_snapshot_date = DATE'{snapshot_date}'
    AND p.mojap_snapshot_date = DATE'{snapshot_date}'
  UNION ALL
  SELECT
    a.case_number,
    a.receipt_date,
    a.year,
    a.quarter,
    a.event,
    a.field_model,
    a.court_code,
    a.order_type,
    a.child_role_id,
    a.dob,
    a.gender,
    a.delete_flag
  FROM
    __temp__.ca_disps_child_case a
"""

pydb.create_temp_table(d4, "ca_disps_all_children")

QueryFailed: SYNTAX_ERROR: line 9:3: Column 'a.year' cannot be resolved. You may need to manually clean the data at location 's3://aws-athena-query-results-593291632749-eu-west-1/tables/e9d40e75-99ac-4c58-8c59-f5620b0349dc' before retrying. Athena will not delete data in your account.